In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.linear_model import MultiTaskLasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
from numpy import linalg as LA

In [3]:
from sklearn.svm import SVC

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.metrics import classification_report

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

'AD SPM Metadata.ipynb'
 ad_tissue_volumes.csv
'Ashraf Haroon Rashid CV.pdf'
'Ashraf Haroon Rashid.pdf'
 cn_tissue_volumes.csv
'CN vs AD feature reduction.ows'
'CN vs MCI vs AD metadata.xlsx'
'Colab Notebooks'
 combinedMetadata.csv
 ev.csv
 ev_df.csv
'Experiment With System commands .ipynb'
'feature_vector_CN_vs_AD (1).csv'
 feature_vector_CN_vs_AD.csv
 Final_Metadata_for_spm_experiment_AD_vs_MCI_1.csv
 Final_Metadata_for_spm_experiment_AD_vs_MCI.csv
 Final_Metadata_for_spm_experiment_CN_vs_MCI.csv
 Final_Metadata_for_spm_experiment.csv
'First class.pdf'
'formulation question design.pdf'
'Hello Python!!!.ipynb'
'Important papers for Alzheimers disease classification - Internship.docx'
'Internship presentation I.pptx'
'Internship report VI materials'
 mci_tissue_volumes.csv
 Metadata_for_spm_experiment.csv
 Metadata_for_spm_experiment_modified.csv
 new_features_CN_vs_AD_MTL.csv
 new_train_data_set_CN_vs_AD.csv
 no_header_shuffled_train_dataset_full_features_AD_vs_MCI.txt
 no_header_shuf

**The SVM has been trained using balanced accuracy in Grid Search**

**Freesurfer dataset CN vs AD**

In [0]:
traindf = pd.read_csv('/content/drive/My Drive/no_header_shuffled_train_dataset_full_features_CN_vs_AD.txt',sep="\t",header=None)
testdf = pd.read_csv('/content/drive/My Drive/no_header_test_dataset_full_features_CN_vs_AD.txt',sep="\t",header=None)

In [0]:
traindf.shape

(80, 92)

In [0]:
df_features = traindf.iloc[:,:91]

df_positive_features = traindf[traindf[91] == 1]
df_positive_features = df_positive_features.iloc[:,:91]

df_negative_features = traindf[traindf[91] == -1]
df_negative_features = df_negative_features.iloc[:,:91]

In [8]:
def fit_kmeans(train_data,n_clusters):
  kmeans = KMeans(n_clusters=n_clusters,init="k-means++",
                  max_iter=500,verbose=1,n_jobs=-1,tol=0.0001,random_state=1,
                  precompute_distances='auto').fit(train_data)
  return kmeans

In [9]:
def prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass):
  zeros = np.zeros([80, n_cluster+1])
  for row in range(0,80):
    if row <= 39:
      zeros[row][0] = 1
    else:
      zeros[row][0] = -1
    
  count = 0
  for idx in positive_features_subclass:
    zeros[count][idx+1] = 1
    count = count + 1
    
  for idx in negative_features_subclass:
    zeros[count][idx+1] = -1
    count = count+1
    
    return zeros

In [10]:
def prepare_multitask_output(class_labels,multitask_matrix,n_cluster):
  num_cols = n_cluster + 1
  multi_task_Y = multitask_matrix.copy()
  count_positive_features = 0
  count_negative_features = 0
  row_index = 0

  for idx in class_labels:
    if idx == 1:
      for col_index in range(0,num_cols):
        multi_task_Y[row_index][col_index] = multitask_matrix[0+count_positive_features][col_index]
     
      count_positive_features = count_positive_features + 1
    else:
      for col_index in range(0,num_cols):
        multi_task_Y[row_index][col_index] = multitask_matrix[40+count_negative_features][col_index]
     
      count_negative_features = count_negative_features + 1
   
    row_index = row_index + 1
    
    return multi_task_Y

In [11]:
def fit_multi_task_lasso(X_train,Y_train,do_cv,best_params = None):
  if do_cv == True:
    pipe = Pipeline(steps=[('scaling', StandardScaler()), ('mtl_lasso',MultiTaskLasso(random_state = 1))])
    kfold = RepeatedKFold(n_splits = 10, n_repeats = 10,random_state = 2)
    pipe_grid = {'mtl_lasso__alpha':[0.0001,0.001,0.01,0.1],'mtl_lasso__tol':[0.01,0.1],'mtl_lasso__max_iter':[15000,20000]}
    mtl_lasso_estimator = GridSearchCV(pipe,pipe_grid,n_jobs = 20,scoring='neg_mean_squared_error',cv = kfold,verbose = 10)
    model = mtl_lasso_estimator.fit(X_train,Y_train)
    
    return model
  
  else:
    mtl_model_lasso = MultiTaskLasso(alpha = best_params['alpha'], max_iter = best_params['max_iter'], tol = best_params['tol'], random_state = 1)
    mtl_model_fit = mtl_model_lasso.fit(X_train,Y_train)
    mtl_coefficients = mtl_model_fit.coef_.T
    
    return mtl_coefficients


In [12]:
def get_lasso_best_params(mtl_model,mtl_model_name):
  best_params = mtl_model.best_params_
  alpha = best_params[mtl_model_name+"__alpha"]
  max_iter = best_params[mtl_model_name+"__max_iter"]
  tol = best_params[mtl_model_name+"__tol"]
  cv_best_params = {'alpha':alpha,'max_iter':max_iter,'tol':tol}
  return cv_best_params

In [13]:
def get_svm_best_params(svm_model,svm_model_name):
  best_params = svm_model.best_params_
  C = best_params[svm_model_name+"__C"]
  gamma = best_params[svm_model_name+"__gamma"]
  kernel = best_params[svm_model_name+"__kernel"]
  cv_best_params = {'C':C,'gamma':gamma,'kernel':kernel}
  return cv_best_params

In [14]:
def get_reduced_feature_list(mtl_coefficients):
  coefficients_l2_norm = LA.norm(mtl_coefficients, ord=2, axis = 1)
  feature_list = []
  count = 0
  
  #print(coefficients_l2_norm)
  for val in coefficients_l2_norm:
    print(val)
    if val > 0:
      print("feature index: "+str(count)+" | L2 norm: "+str(val))
      feature_list.append(count)
      
    count = count + 1
    
    
  return feature_list  

In [15]:
def fit_svm(X_train,Y_train,cv,best_params=None):
  if cv == True:
    pipe = Pipeline(steps=[('scaling', StandardScaler()), ('classifier',SVC())])
    pipe_grid = {'classifier__C':[0.0001,0.001,0.01,0.1,1,10,100,1000],'classifier__gamma':[0.0001,0.001,0.01,10,100,1000],'classifier__kernel':['linear','rbf','poly','sigmoid']}
    kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10,random_state = 2)
    svm_estimator = GridSearchCV(pipe,pipe_grid,n_jobs = 20,scoring='balanced_accuracy',cv = kfold,verbose = 10)
    svm_model = svm_estimator.fit(X_train.values,Y_train.values.ravel())
    
    return svm_model
  else:
    pipe = Pipeline(steps=[('scaling', StandardScaler()), ('classifier',SVC(C=best_params['C'],gamma=best_params['gamma'],kernel=best_params['kernel']))])
    svm_model_fit = pipe.fit(X_train.values,Y_train.values.ravel())
    
    return svm_model_fit

In [16]:
def predict_svm(svm_model,Y_test):
  return svm_model.predict(Y_test.values)

In [0]:
kmeans_clusters = [2,3,4,5]

Number of clusters = 2

In [0]:
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 2
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 304.02165808213425
start iteration
done sorting
end inner loop
Iteration 1, inertia 301.82287890014175
start iteration
done sorting
end inner loop
Iteration 2, inertia 301.82287890014175
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 327.85471548753026
start iteration
done sorting
end inner loop
Iteration 1, inertia 305.5242036798213
start iteration
done sorting
end inner loop
Iteration 2, inertia 303.5833799758469
start iteration
done sorting
end inner loop
Iteration 3, inertia 302.96474090937784
start iteration
done sorting
end inner loop
Iteration 4, inertia 301.28176830345086
start iteration
done sorting
end inner loop
Iteration 5, inertia 301.28176830345086
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner l

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    6.6s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:    6.7s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    6.8s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:    7.0s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:    7.4s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.0
0.0
0.04623909958897832
feature index: 2 | L2 norm: 0.04623909958897832
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.01625631502007442
feature index: 18 | L2 norm: 0.01625631502007442
0.009703175281397683
feature index: 19 | L2 norm: 0.009703175281397683
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0853229433399939
feature index: 28 | L2 norm: 0.0853229433399939
0.0
0.04197849743194355
feature index: 30 | L2 norm: 0.04197849743194355
0.0
0.05138288377663256
feature index: 32 | L2 norm: 0.05138288377663256
0.0
0.0
0.0
0.0
0.05964318502905892
feature index: 37 | L2 norm: 0.05964318502905892
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3564739243612373
feature index: 46 | L2 norm: 0.3564739243612373
0.0
0.0
0.0
0.0
0.0
0.0
0.01235800520509824
feature index: 53 | L2 norm: 0.01235800520509824
0.10970707857892487
feature index: 54 | L2 norm: 0.10970707857892487
0.1712520871438011
feature index: 55 | L2 norm: 0.

[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:   11.6s
[Parallel(n_jobs=20)]: Batch computation too slow (20.9302s.) Setting batch_size=516.
[Parallel(n_jobs=20)]: Done 5200 tasks      | elapsed:   31.6s
[Parallel(n_jobs=20)]: Batch computation too slow (18.3336s.) Setting batch_size=258.
[Parallel(n_jobs=20)]: Done 10876 tasks      | elapsed:   48.6s
[Parallel(n_jobs=20)]: Done 16000 out of 16000 | elapsed:  2.4min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


******** SVM best params **********
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

***** accuracy *****
0.8

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.60      0.75        10
           1       0.71      1.00      0.83        10

   micro avg       0.80      0.80      0.80        20
   macro avg       0.86      0.80      0.79        20
weighted avg       0.86      0.80      0.79        20

Number of clusters: 3
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 268.2265699515411
start iteration
done sorting
end inner loop
Iteration 1, inertia 262.57584370923274
start iteration
done sorting
end inner loop
Iteration 2, inertia 262.16698364759065
start iteration
done sorting
end inner loop
Iteration 3, inertia 261.8846999551871
start iteration
done sorting
end inner loop
Iteration 4, inertia 261.68159598956885
start iteration
done sorting
end inner loop
Iteration 5, inertia 261

...

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    7.2s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    9.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   10.5s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   10.7s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   11.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   11.6s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   11.7s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   12.5s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:   15.2s
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:   18.1s
[Paralle

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.010255200381800837
feature index: 0 | L2 norm: 0.010255200381800837
0.0
0.048305400488582464
feature index: 2 | L2 norm: 0.048305400488582464
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.006715687676499656
feature index: 18 | L2 norm: 0.006715687676499656
0.02489806478958442
feature index: 19 | L2 norm: 0.02489806478958442
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.010463946040010403
feature index: 28 | L2 norm: 0.010463946040010403
0.0
0.04159142652318266
feature index: 30 | L2 norm: 0.04159142652318266
0.0
0.10094325708028563
feature index: 32 | L2 norm: 0.10094325708028563
0.0
0.0
0.0
0.0
0.016780310097199724
feature index: 37 | L2 norm: 0.016780310097199724
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3242572245806465
feature index: 46 | L2 norm: 0.3242572245806465
0.0
0.0
0.0
0.0
0.0
0.0
0.013581046032392782
feature index: 53 | L2 norm: 0.013581046032392782
0.11404316546060819
feature index: 54 | L2

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0434s.) Setting batch_size=8.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 424 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Done 560 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 696 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 848 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 1000 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1168 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done 1336 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done 1520 tasks      | 

******** SVM best params **********
{'C': 0.0001, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.85

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.70      0.82        10
           1       0.77      1.00      0.87        10

   micro avg       0.85      0.85      0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20



[Parallel(n_jobs=20)]: Done 16000 out of 16000 | elapsed:  2.3min finished


In [0]:
# mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
# best_params = get_lasso_best_params(mtl_model,'mtl_lasso')

l = [2, 18, 19, 28, 30, 32, 37, 46, 53, 54, 55, 58, 68, 70, 72, 74, 77, 79, 82, 84]
testdf[l].head()

,2,18,19,28,30,32,37,46,53,54,55,58,68,70,72,74,77,79,82,84
0,5.027,3.543,5.395,4.697,5.503,5.496,0.000884,0.000631,0.005711,0.002416,0.009310,0.008322,0.004072,0.001252,0.004679,0.008439,0.002305,0.009867,0.003612,0.001401
1,5.392,3.389,5.169,3.928,4.928,4.885,0.000870,0.000557,0.006056,0.002492,0.009160,0.008097,0.003995,0.001166,0.005191,0.010718,0.001919,0.011192,0.004081,0.001612
2,5.563,3.743,5.933,4.372,4.964,5.851,0.000460,0.000597,0.005805,0.002573,0.009448,0.008546,0.004236,0.001548,0.005039,0.009145,0.001966,0.012044,0.004698,0.001608
3,5.609,3.804,4.594,4.470,5.095,4.958,0.001197,0.000416,0.005016,0.002158,0.008067,0.008331,0.003706,0.000945,0.005107,0.008726,0.001857,0.009996,0.003757,0.001360
4,5.396,3.557,5.406,4.116,5.127,5.044,0.001446,0.000540,0.005635,0.002422,0.009093,0.008259,0.004160,0.001239,0.005170,0.008515,0.001929,0.010434,0.004277,0.001660


In [0]:
kmeans_clusters = [6,7,8,9,10]

for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 6
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 241.00705852349225
start iteration
done sorting
end inner loop
Iteration 1, inertia 220.2248208886988
start iteration
done sorting
end inner loop
Iteration 2, inertia 215.62137273631473
start iteration
done sorting
end inner loop
Iteration 3, inertia 214.38325871740403
start iteration
done sorting
end inner loop
Iteration 4, inertia 210.76747334505194
start iteration
done sorting
end inner loop
Iteration 5, inertia 210.76747334505194
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 231.87962139674354
start iteration
done sorting
end inner loop
Iteration 1, inertia 221.69422099469116
start iteration
done sorting
end inner loop
Iteration 2, inertia 217.78162802125726
start iteration
done sorting
end inner loop
Iteration 3, inertia 215.26660350455313
start iteration
done sorting
end in

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    8.7s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   12.8s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   17.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   20.3s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   22.0s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   23.7s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   25.8s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   26.9s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   27.2s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   27.5s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   28.3s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   28.5s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   28.6s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   29.3s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.02184788639751063
feature index: 0 | L2 norm: 0.02184788639751063
0.0
0.02708914377404559
feature index: 2 | L2 norm: 0.02708914377404559
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.010234214701564198
feature index: 18 | L2 norm: 0.010234214701564198
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.06158979201466297
feature index: 28 | L2 norm: 0.06158979201466297
0.0
0.030028331586409888
feature index: 30 | L2 norm: 0.030028331586409888
0.0
0.09149135931557026
feature index: 32 | L2 norm: 0.09149135931557026
0.0
0.015141058523180198
feature index: 34 | L2 norm: 0.015141058523180198
0.0
0.0
0.04086150050245496
feature index: 37 | L2 norm: 0.04086150050245496
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3296917566187619
feature index: 46 | L2 norm: 0.3296917566187619
0.0
0.0
0.0
0.0
0.0
0.0
0.01849754557788535
feature index: 53 | L2 norm: 0.01849754557788535
0.09303131523502985
feature index: 54 | L2 norm: 

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 1024 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 1252 tasks      | elapsed:    5.2s
[Parallel(n_jobs=20)]: Done 1480 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done 1732 tasks      | elapsed:    7.0s
[Parallel(n_jobs=20)]: Done 1984 tasks      | elapsed:    7.9s
[Parallel(n_jobs=20)]: Done 2260 tasks      | elapsed:    9.0s
[Parallel(n_jobs=20)]: Done 2536 tasks      | elapsed:   10.0s
[Parallel(n_jobs=20)]: Done 2836 tasks      | elapsed:   11.2s
[Parallel(n_jobs=20)]: Done 3136 tasks      | elapsed:   12.4s

******** SVM best params **********
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

***** accuracy *****
0.75

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.50      0.67        10
           1       0.67      1.00      0.80        10

   micro avg       0.75      0.75      0.75        20
   macro avg       0.83      0.75      0.73        20
weighted avg       0.83      0.75      0.73        20

Number of clusters: 7
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 237.8536201465153
start iteration
done sorting
end inner loop
Iteration 1, inertia 217.5389405222232
start iteration
done sorting
end inner loop
Iteration 2, inertia 213.08584080321373
start iteration
done sorting
end inner loop
Iteration 3, inertia 209.77258558380817
start iteration
done sorting
end inner loop
Iteration 4, inertia 206.15680021145607
start iteration
done sorting
end inner loop
Iteration 5, inertia 

...

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    8.1s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   16.0s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   17.3s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   17.8s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   18.1s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   18.4s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   18.7s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   19.1s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   22.1s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.027626989870314353
feature index: 2 | L2 norm: 0.027626989870314353
0.0
0.023033084363882503
feature index: 4 | L2 norm: 0.023033084363882503
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.00634527325752512
feature index: 18 | L2 norm: 0.00634527325752512
0.0032572136124263185
feature index: 19 | L2 norm: 0.0032572136124263185
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.022989344027100556
feature index: 28 | L2 norm: 0.022989344027100556
0.0
0.03255742231915084
feature index: 30 | L2 norm: 0.03255742231915084
0.0
0.08540029106559485
feature index: 32 | L2 norm: 0.08540029106559485
0.0
0.053347672306025294
feature index: 34 | L2 norm: 0.053347672306025294
0.0
0.0
0.008268459628869483
feature index: 37 | L2 norm: 0.008268459628869483
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.30844106121376913
feature index: 46 | L2 norm: 0.30844106121376913
0.0
0.0
0.0
0.0
0.0
0.0
0.03498210419476641
feature index: 53 | L2

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0435s.) Setting batch_size=8.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 424 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 560 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 696 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 848 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 1000 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 1168 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 1336

******** SVM best params **********
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

***** accuracy *****
0.75

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.50      0.67        10
           1       0.67      1.00      0.80        10

   micro avg       0.75      0.75      0.75        20
   macro avg       0.83      0.75      0.73        20
weighted avg       0.83      0.75      0.73        20



[Parallel(n_jobs=20)]: Done 16000 out of 16000 | elapsed:  1.2min finished


In [0]:
kmeans_clusters = [11,12,13,14,15]

for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 11
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 177.8714701683508
start iteration
done sorting
end inner loop
Iteration 1, inertia 165.77966780731214
start iteration
done sorting
end inner loop
Iteration 2, inertia 165.20871222056596
start iteration
done sorting
end inner loop
Iteration 3, inertia 165.20871222056596
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 175.6458453051228
start iteration
done sorting
end inner loop
Iteration 1, inertia 168.47567057481774
start iteration
done sorting
end inner loop
Iteration 2, inertia 167.5773661090987
start iteration
done sorting
end inner loop
Iteration 3, inertia 166.6532336786329
start iteration
done sorting
end inner loop
Iteration 4, inertia 165.84122028526758
start iteration
done sorting
end inner loop
Iteration 5, inertia 165.84122028526758
center shift 0.000000e+00 within tole

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   14.3s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   20.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   22.4s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   25.4s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   28.1s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   31.8s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   33.6s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   34.0s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   34.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   34.6s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   35.0s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   35.3s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   38.5s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.04114737002666367
feature index: 2 | L2 norm: 0.04114737002666367
0.0
0.002868602487240925
feature index: 4 | L2 norm: 0.002868602487240925
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.006682460085182008
feature index: 18 | L2 norm: 0.006682460085182008
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.017880745719485305
feature index: 28 | L2 norm: 0.017880745719485305
0.0
0.04561531819667206
feature index: 30 | L2 norm: 0.04561531819667206
0.0
0.06994620426503774
feature index: 32 | L2 norm: 0.06994620426503774
0.0
0.029419739382888235
feature index: 34 | L2 norm: 0.029419739382888235
0.0
0.0
0.010878296742008816
feature index: 37 | L2 norm: 0.010878296742008816
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3211752140985653
feature index: 46 | L2 norm: 0.3211752140985653
0.0
0.0
0.0
0.0
0.0
0.0
0.03728491636256271
feature index: 53 | L2 norm: 0.03728491636256271
0.10160287223378976
feature index: 54 | L2 n

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:    4.0s
[Parallel(n_jobs=20)]: Done 1024 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1252 tasks      | elapsed:    5.7s
[Parallel(n_jobs=20)]: Done 1480 tasks      | elapsed:    6.5s
[Parallel(n_jobs=20)]: Done 1732 tasks      | elapsed:    7.4s
[Parallel(n_jobs=20)]: Done 1984 tasks      | elapsed:    8.3s
[Parallel(n_jobs=20)]: Done 2260 tasks      | elapsed:    9.2s
[Parallel(n_jobs=20)]: Done 2536 tasks      | elapsed:   10.4s
[Parallel(n_jobs=20)]: Done 2836 tasks      | elapsed:   11.5s


******** SVM best params **********
{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

***** accuracy *****
0.85

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.70      0.82        10
           1       0.77      1.00      0.87        10

   micro avg       0.85      0.85      0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20

Number of clusters: 12
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 173.0700178641589
start iteration
done sorting
end inner loop
Iteration 1, inertia 161.17139661504223
start iteration
done sorting
end inner loop
Iteration 2, inertia 160.55914416048807
start iteration
done sorting
end inner loop
Iteration 3, inertia 160.55914416048807
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 

...

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0358s.) Setting batch_size=10.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done 16

******** SVM best params **********
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

***** accuracy *****
0.85

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.70      0.82        10
           1       0.77      1.00      0.87        10

   micro avg       0.85      0.85      0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20

Number of clusters: 14
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 162.35956765578314
start iteration
done sorting
end inner loop
Iteration 1, inertia 154.40046414621654
start iteration
done sorting
end inner loop
Iteration 2, inertia 153.78821169166238
start iteration
done sorting
end inner loop
Iteration 3, inertia 153.78821169166238
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    6.5s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    9.5s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   13.1s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   15.7s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   20.0s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   22.9s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   23.2s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   23.5s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   23.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   24.3s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   24.6s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   28.6s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:   33.2s
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:   39.8s
[Paralle

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.015722579155921276
feature index: 0 | L2 norm: 0.015722579155921276
0.0
0.026977281162080872
feature index: 2 | L2 norm: 0.026977281162080872
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.005591141503258593
feature index: 18 | L2 norm: 0.005591141503258593
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08927177357047822
feature index: 28 | L2 norm: 0.08927177357047822
0.0
0.03528899203697236
feature index: 30 | L2 norm: 0.03528899203697236
0.0
0.07328809339181948
feature index: 32 | L2 norm: 0.07328809339181948
0.0
0.03315765727633493
feature index: 34 | L2 norm: 0.03315765727633493
0.0
0.0
0.05210510747155972
feature index: 37 | L2 norm: 0.05210510747155972
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.30558833688242554
feature index: 46 | L2 norm: 0.30558833688242554
0.0
0.0
0.0
0.0
0.0
0.0
0.0291387973276011
feature index: 53 | L2 norm: 0.0291387973276011
0.08729066072923278
feature index: 54 | L2 norm: 

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0305s.) Setting batch_size=12.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 1024 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 1252 tasks      | elapsed:    5.7s
[Parallel(n_jobs=20)]: Done 1480 tasks      | elapsed:    6.6s
[Parallel(n_jobs=20)]: Done 1732 tasks      | elapsed:    7.4s
[Parallel(n_jobs=20)]: Done 1

In [0]:
kmeans_clusters = [14,15,16,17,18,19,20]

for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 14
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 162.35956765578314
start iteration
done sorting
end inner loop
Iteration 1, inertia 154.40046414621654
start iteration
done sorting
end inner loop
Iteration 2, inertia 153.78821169166238
start iteration
done sorting
end inner loop
Iteration 3, inertia 153.78821169166238
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 157.79159581389194
start iteration
done sorting
end inner loop
Iteration 1, inertia 152.56129036918819
start iteration
done sorting
end inner loop
Iteration 2, inertia 152.56129036918819
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 161.8313898991609
start iteration
done sorting
end inner loop
Iteration 1, inertia 155.69788586920066
start iteration
done sorting
end inner

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    8.6s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   15.3s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   20.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   23.4s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   27.3s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   29.9s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   34.0s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   36.8s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   37.2s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   37.5s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   37.8s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   38.1s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   38.4s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   42.3s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.015722579155921276
feature index: 0 | L2 norm: 0.015722579155921276
0.0
0.026977281162080872
feature index: 2 | L2 norm: 0.026977281162080872
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.005591141503258593
feature index: 18 | L2 norm: 0.005591141503258593
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08927177357047822
feature index: 28 | L2 norm: 0.08927177357047822
0.0
0.03528899203697236
feature index: 30 | L2 norm: 0.03528899203697236
0.0
0.07328809339181948
feature index: 32 | L2 norm: 0.07328809339181948
0.0
0.03315765727633493
feature index: 34 | L2 norm: 0.03315765727633493
0.0
0.0
0.05210510747155972
feature index: 37 | L2 norm: 0.05210510747155972
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.30558833688242554
feature index: 46 | L2 norm: 0.30558833688242554
0.0
0.0
0.0
0.0
0.0
0.0
0.0291387973276011
feature index: 53 | L2 norm: 0.0291387973276011
0.08729066072923278
feature index: 54 | L2 norm: 

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0390s.) Setting batch_size=10.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    3.8s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    5.6s
[Parallel(n_jobs=20)]: Done 1890 tasks      

******** SVM best params **********
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

***** accuracy *****
0.8

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.60      0.75        10
           1       0.71      1.00      0.83        10

   micro avg       0.80      0.80      0.80        20
   macro avg       0.86      0.80      0.79        20
weighted avg       0.86      0.80      0.79        20

Number of clusters: 15
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 154.05137727547046
start iteration
done sorting
end inner loop
Iteration 1, inertia 148.41847420931316
start iteration
done sorting
end inner loop
Iteration 2, inertia 147.806221754759
start iteration
done sorting
end inner loop
Iteration 3, inertia 147.806221754759
center shift 0.000000e+00 within tolerance 7.030326e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 158.0929

...

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    8.6s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   12.2s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   16.9s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   19.8s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   26.0s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   28.8s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   29.4s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   29.8s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   30.2s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   30.5s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   30.8s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   36.2s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:   42.9s
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:   51.3s
[Paralle

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.0030242551070937024
feature index: 0 | L2 norm: 0.0030242551070937024
0.0
0.029445748756969356
feature index: 2 | L2 norm: 0.029445748756969356
0.0
0.017765746468798194
feature index: 4 | L2 norm: 0.017765746468798194
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.007059354645654639
feature index: 19 | L2 norm: 0.007059354645654639
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0704459866690039
feature index: 28 | L2 norm: 0.0704459866690039
0.0
0.028034984280607085
feature index: 30 | L2 norm: 0.028034984280607085
0.0
0.07291816955690235
feature index: 32 | L2 norm: 0.07291816955690235
0.0
0.02812287046666742
feature index: 34 | L2 norm: 0.02812287046666742
0.0
0.0
0.04422003104636383
feature index: 37 | L2 norm: 0.04422003104636383
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3108696684608705
feature index: 46 | L2 norm: 0.3108696684608705
0.0
0.0
0.0
0.0
0.0
0.0
0.027586808969050858
feature index: 53 | L2 norm: 

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0360s.) Setting batch_size=10.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.6s
[Parallel(n_jobs=20)]: Done 16

******** SVM best params **********
{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

***** accuracy *****
0.8

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.60      0.75        10
           1       0.71      1.00      0.83        10

   micro avg       0.80      0.80      0.80        20
   macro avg       0.86      0.80      0.79        20
weighted avg       0.86      0.80      0.79        20



[Parallel(n_jobs=20)]: Done 16000 out of 16000 | elapsed:  1.1min finished


In [0]:
coefficients_l2_norm = LA.norm(mtl_coefficients, ord=2, axis = 1)

In [0]:
coefficients_l2_norm

array([0.        , 0.        , 0.0462391 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01625632, 0.00970318,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.08532294, 0.        ,
       0.0419785 , 0.        , 0.05138288, 0.        , 0.        ,
       0.        , 0.        , 0.05964319, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.35647392, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01235801, 0.10970708,
       0.17125209, 0.        , 0.        , 0.03668195, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00402973, 0.        ,
       0.00521146, 0.        , 0.08284804, 0.        , 0.00593

In [0]:
f = get_reduced_feature_list(mtl_coefficients)

[0.         0.         0.0462391  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.01625632 0.00970318 0.         0.         0.         0.
 0.         0.         0.         0.         0.08532294 0.
 0.0419785  0.         0.05138288 0.         0.         0.
 0.         0.05964319 0.         0.         0.         0.
 0.         0.         0.         0.         0.35647392 0.
 0.         0.         0.         0.         0.         0.01235801
 0.10970708 0.17125209 0.         0.         0.03668195 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.00402973 0.         0.00521146 0.
 0.08284804 0.         0.00593703 0.         0.         0.06935887
 0.         0.04132584 0.         0.         0.11755934 0.
 0.03033245 0.         0.         0.         0.         0.
 0.        ]
0.0
0.0
0.04623909958897832
feature index: 2 | L2 norm: 0.04623909958897832
0.0
0.0
0.

In [0]:
len(f)

20

In [0]:
count = 0
for val in coefficients_l2_norm:
    
    print("norm: "+str(val))
    if val > 0:
      print("feature index: "+str(count)+" | L2 norm: "+str(val))
    count = count + 1

norm: 0.0
norm: 0.0
norm: 0.04623909958897832
feature index: 2 | L2 norm: 0.04623909958897832
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.01625631502007442
feature index: 18 | L2 norm: 0.01625631502007442
norm: 0.009703175281397683
feature index: 19 | L2 norm: 0.009703175281397683
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0853229433399939
feature index: 28 | L2 norm: 0.0853229433399939
norm: 0.0
norm: 0.04197849743194355
feature index: 30 | L2 norm: 0.04197849743194355
norm: 0.0
norm: 0.05138288377663256
feature index: 32 | L2 norm: 0.05138288377663256
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.05964318502905892
feature index: 37 | L2 norm: 0.05964318502905892
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.0
norm: 0.3564739243612373
feature index: 46 | L2 norm: 0.3564739243612373
norm: 0.0
nor

**SPM GM features** 

In [0]:
traindf = pd.read_csv('/content/drive/My Drive/train_dataset_GM.txt',sep="\t",header=None)
testdf = pd.read_csv('/content/drive/My Drive/test_dataset_GM.txt',sep="\t",header=None)

In [0]:
traindf.shape

(80, 26525)

In [0]:
traindf.head()

,0,1,2,3,4,5,6,7,8,9,...,26515,26516,26517,26518,26519,26520,26521,26522,26523,26524
0,0.149342,0.141262,0.154486,0.155960,0.228231,0.223465,0.212868,0.223435,0.233199,0.239515,...,0.487606,0.402472,0.470469,0.492342,0.447811,0.452680,0.478292,0.409888,0.418728,1
1,0.167770,0.163206,0.177834,0.182207,0.228982,0.224994,0.214814,0.225674,0.235816,0.243392,...,0.327517,0.341536,0.289619,0.306006,0.283949,0.277206,0.294583,0.264952,0.263669,-1
2,0.116128,0.116981,0.116123,0.123265,0.126397,0.131521,0.131606,0.120879,0.128885,0.138267,...,0.313920,0.260362,0.245103,0.265213,0.249879,0.209828,0.229962,0.202282,0.171799,-1
3,0.197550,0.187252,0.196843,0.197337,0.288350,0.278284,0.257781,0.279684,0.296917,0.307673,...,0.429024,0.491395,0.395448,0.429982,0.410249,0.376424,0.418082,0.381288,0.345195,1
4,0.209001,0.195105,0.213437,0.212073,0.297953,0.291831,0.271301,0.269353,0.292699,0.310448,...,0.451753,0.328764,0.499725,0.539849,0.474824,0.484044,0.529970,0.436863,0.450022,1


In [0]:
testdf.head()

,0,1,2,3,4,5,6,7,8,9,...,26515,26516,26517,26518,26519,26520,26521,26522,26523,26524
0,0.198580,0.184662,0.212292,0.208186,0.255970,0.251118,0.238139,0.261131,0.273553,0.282858,...,0.224005,0.427904,0.383127,0.368602,0.306470,0.322693,0.318492,0.262402,0.266154,1.0
1,0.160201,0.155787,0.159400,0.161453,0.218727,0.219491,0.211478,0.199956,0.216149,0.229577,...,0.347078,0.389728,0.295442,0.304574,0.254922,0.259775,0.275212,0.218662,0.218398,1.0
2,0.165057,0.154998,0.173301,0.169807,0.234221,0.228527,0.214254,0.234448,0.246940,0.254255,...,0.409330,0.468593,0.391454,0.410680,0.391659,0.362737,0.389665,0.349847,0.320006,1.0
3,0.211609,0.199756,0.218534,0.213680,0.302603,0.293391,0.276890,0.311815,0.321259,0.324936,...,0.348964,0.274678,0.463872,0.489795,0.469748,0.448513,0.479920,0.440437,0.414039,1.0
4,0.180137,0.178925,0.175589,0.182635,0.236870,0.243967,0.241028,0.213835,0.228985,0.245951,...,0.341367,0.423518,0.337465,0.350439,0.323453,0.311551,0.331755,0.288338,0.275355,1.0


In [0]:
df_features = traindf.iloc[:,:26524]

df_positive_features = traindf[traindf[26524] == 1]
df_positive_features = df_positive_features.iloc[:,:26524]

df_negative_features = traindf[traindf[26524] == -1]
df_negative_features = df_negative_features.iloc[:,:26524]

In [0]:
df_negative_features.head()

,0,1,2,3,4,5,6,7,8,9,...,26514,26515,26516,26517,26518,26519,26520,26521,26522,26523
1,0.167770,0.163206,0.177834,0.182207,0.228982,0.224994,0.214814,0.225674,0.235816,0.243392,...,0.288052,0.327517,0.341536,0.289619,0.306006,0.283949,0.277206,0.294583,0.264952,0.263669
2,0.116128,0.116981,0.116123,0.123265,0.126397,0.131521,0.131606,0.120879,0.128885,0.138267,...,0.264580,0.313920,0.260362,0.245103,0.265213,0.249879,0.209828,0.229962,0.202282,0.171799
9,0.149159,0.146130,0.154473,0.155114,0.213627,0.205604,0.195629,0.229577,0.235051,0.234270,...,0.235640,0.277406,0.360921,0.368912,0.399523,0.342998,0.338799,0.368463,0.296146,0.296594
11,0.115402,0.102697,0.113649,0.106815,0.216250,0.209594,0.193179,0.204195,0.208253,0.211651,...,0.302226,0.345343,0.315175,0.303299,0.337025,0.292660,0.301226,0.336781,0.281426,0.294973
13,0.176985,0.171329,0.178718,0.182136,0.249687,0.249378,0.235213,0.216041,0.233394,0.251971,...,0.424450,0.466931,0.338142,0.413316,0.424294,0.384127,0.388057,0.402672,0.340620,0.347021


In [0]:
kmeans_clusters = [2,4,6,8,10]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[26524],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[26524]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[26524]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[26524]].values,svm_predict)
  report = classification_report(testdf[[26524]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 2
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 7240.18875429888
start iteration
done sorting
end inner loop
Iteration 1, inertia 6799.240239474352
start iteration
done sorting
end inner loop
Iteration 2, inertia 6617.775316835931
start iteration
done sorting
end inner loop
Iteration 3, inertia 6588.290491910007
start iteration
done sorting
end inner loop
Iteration 4, inertia 6588.290491910007
center shift 0.000000e+00 within tolerance 6.223524e-07
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 6760.803500550619
start iteration
done sorting
end inner loop
Iteration 1, inertia 6606.153450972924
start iteration
done sorting
end inner loop
Iteration 2, inertia 6594.991590744383
start iteration
done sorting
end inner loop
Iteration 3, inertia 6594.991590744383
center shift 0.000000e+00 within tolerance 6.223524e-07
Initialization complete
start iteration
done sorting
end inner loop
Iter

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   16.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   25.1s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   25.5s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   30.7s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   35.1s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   36.5s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   41.5s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   46.0s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   50.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   54.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   59.0s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:  1.1min
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:  1.1min
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0428s.) Setting batch_size=8.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 424 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 560 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 696 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 848 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 1000 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done 1168 tasks      | elapsed:    5.7s
[Parallel(n_jobs=20)]: Done 1336

******** SVM best params **********
{'C': 0.1, 'gamma': 1000, 'kernel': 'sigmoid'}

***** accuracy *****
0.65

***** Classification report ********
              precision    recall  f1-score   support

        -1.0       0.67      0.60      0.63        10
         1.0       0.64      0.70      0.67        10

   micro avg       0.65      0.65      0.65        20
   macro avg       0.65      0.65      0.65        20
weighted avg       0.65      0.65      0.65        20

Number of clusters: 4
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 4711.565486872811
start iteration
done sorting
end inner loop
Iteration 1, inertia 4588.380472471599
start iteration
done sorting
end inner loop
Iteration 2, inertia 4570.4036518623
start iteration
done sorting
end inner loop
Iteration 3, inertia 4549.555767540952
start iteration
done sorting
end inner loop
Iteration 4, inertia 4549.555767540952
center shift 0.000000e+00 within tolerance 6.223524e-07
Initializa

...

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   11.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   22.7s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   24.5s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   36.2s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   45.7s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   50.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:  1.0min
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:  1.2min
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:  1.4min
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:  1.5min
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  1.7min
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:  1.9min
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:  2.0min
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0310s.) Setting batch_size=12.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0220s.) Setting batch_size=6.
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 784 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 892 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done 1006 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 1120 tasks      | elapsed:    5.5s
[Parallel(

******** SVM best params **********
{'C': 0.1, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.55

***** Classification report ********
              precision    recall  f1-score   support

        -1.0       0.56      0.50      0.53        10
         1.0       0.55      0.60      0.57        10

   micro avg       0.55      0.55      0.55        20
   macro avg       0.55      0.55      0.55        20
weighted avg       0.55      0.55      0.55        20



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed:  1.4min finished


**Freesurfer dataset CN vs MCI**

In [0]:
traindf = pd.read_csv('/content/drive/My Drive/no_header_shuffled_train_dataset_full_features_CN_vs_MCI.txt',sep="\t",header=None)
testdf = pd.read_csv('/content/drive/My Drive/no_header_test_dataset_full_features_CN_vs_MCI.txt',sep="\t",header=None)

In [0]:
df_features = traindf.iloc[:,:91]

df_positive_features = traindf[traindf[91] == 1]
df_positive_features = df_positive_features.iloc[:,:91]

df_negative_features = traindf[traindf[91] == -1]
df_negative_features = df_negative_features.iloc[:,:91]

In [0]:
kmeans_clusters = [2,3,4,5,6,7,8,9,10]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 2
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 276.9434911876759
start iteration
done sorting
end inner loop
Iteration 1, inertia 254.26889379248988
start iteration
done sorting
end inner loop
Iteration 2, inertia 251.23450219386433
start iteration
done sorting
end inner loop
Iteration 3, inertia 250.60373721150734
start iteration
done sorting
end inner loop
Iteration 4, inertia 250.27632701102198
start iteration
done sorting
end inner loop
Iteration 5, inertia 250.27632701102198
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 269.78785298284055
start iteration
done sorting
end inner loop
Iteration 1, inertia 263.754508185296
start iteration
done sorting
end inner loop
Iteration 2, inertia 259.88710239010345
start iteration
done sorting
end inner loop
Iteration 3, inertia 258.354492896616
start iteration
done sorting
end inner 

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0314s.) Setting batch_size=12.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0236s.) Setting batch_size=6.
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Batch computation too slow (3.2900s.) Setting batch_size=3.
[Parallel(n_jobs=20)]: Done 814 tasks      | elapsed:    6.8s
[Parallel(n_jobs=20)]: Done 1006 tasks      | elapsed:    8.1s
[Parallel(n_jobs=20)]: Done 1102 tasks      | elapse

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.1}

0.0
0.0
0.1419592549541853
feature index: 2 | L2 norm: 0.1419592549541853
0.0004497338815787809
feature index: 3 | L2 norm: 0.0004497338815787809
0.07175576422499619
feature index: 4 | L2 norm: 0.07175576422499619
0.022964341325297784
feature index: 5 | L2 norm: 0.022964341325297784
0.0
0.0
0.012841729373026552
feature index: 8 | L2 norm: 0.012841729373026552
0.0
0.0
0.0
0.0
0.0
0.010176125473776623
feature index: 14 | L2 norm: 0.010176125473776623
0.0
0.06974228089789299
feature index: 16 | L2 norm: 0.06974228089789299
0.03336082090496858
feature index: 17 | L2 norm: 0.03336082090496858
0.0
0.027767358253009845
feature index: 19 | L2 norm: 0.027767358253009845
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.007490036184947172
feature index: 29 | L2 norm: 0.007490036184947172
0.09807653244850367
feature index: 30 | L2 norm: 0.09807653244850367
0.0
0.004612944555884776
feature index: 32 | L2 norm: 0.004612944555884

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.7s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.5s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    9.3s
[

******** SVM best params **********
{'C': 0.0001, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.8421052631578947

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.80      0.89      0.84         9
           1       0.89      0.80      0.84        10

   micro avg       0.84      0.84      0.84        19
   macro avg       0.84      0.84      0.84        19
weighted avg       0.85      0.84      0.84        19

Number of clusters: 3
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 243.29431393757605
start iteration
done sorting
end inner loop
Iteration 1, inertia 225.80381925408386
start iteration
done sorting
end inner loop
Iteration 2, inertia 221.8485103054484
start iteration
done sorting
end inner loop
Iteration 3, inertia 219.8544075641987
start iteration
done sorting
end inner loop
Iteration 4, inertia 218.83515441037926
start iteration
done sorting
end inner loop
It

...

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0830s.) Setting batch_size=4.
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 424 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Batch computation too slow (2.3629s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done 528 tasks      | elapsed:    8.9s
[Parallel(n_jobs=20)]: Done 644 tasks      | elapsed:   12.2s
[Parallel(n_jobs=20)]: Batch computation too slow (3.7630s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done 686 tasks      | elapsed:   12.9s
[Parallel(n_jobs=20)]: Done 717 tasks      | elapsed:   13.2s
[Parallel(n_jobs=20)]: Done 744 tasks      | elapsed:   13.4s
[Parall

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17889100432629412
feature index: 2 | L2 norm: 0.17889100432629412
0.0
0.04749070253220994
feature index: 4 | L2 norm: 0.04749070253220994
0.05624394188411126
feature index: 5 | L2 norm: 0.05624394188411126
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.013884649814798243
feature index: 14 | L2 norm: 0.013884649814798243
0.0
0.07207592691299679
feature index: 16 | L2 norm: 0.07207592691299679
0.0
0.0
0.019363085376379967
feature index: 19 | L2 norm: 0.019363085376379967
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.021997911658248253
feature index: 29 | L2 norm: 0.021997911658248253
0.11416710723747445
feature index: 30 | L2 norm: 0.11416710723747445
0.0
0.02859929680214587
feature index: 32 | L2 norm: 0.02859929680214587
0.0
0.07345229425167107
feature index: 34 | L2 norm: 0.07345229425167107
0.0
0.0
0.0
0.0
0.0
0.1233442143637123
feature index: 40 | L2 norm: 0.1233442143637123
0.0
0.0
0.0
0.0
0.0
0.30415790413161

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0418s.) Setting batch_size=8.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 424 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 560 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done 696 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 848 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 1000 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 1168 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1336 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 1520 tasks      | 

******** SVM best params **********
{'C': 0.01, 'gamma': 0.0001, 'kernel': 'linear'}

***** accuracy *****
0.8947368421052632

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.89      0.89      0.89         9
           1       0.90      0.90      0.90        10

   micro avg       0.89      0.89      0.89        19
   macro avg       0.89      0.89      0.89        19
weighted avg       0.89      0.89      0.89        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed:  9.9min finished


In [0]:
kmeans_clusters = [11,12,13,14,15,16,17,18,19,20]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 11
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 149.13496866485613
start iteration
done sorting
end inner loop
Iteration 1, inertia 146.96959583984236
start iteration
done sorting
end inner loop
Iteration 2, inertia 144.76248145803285
start iteration
done sorting
end inner loop
Iteration 3, inertia 144.11933105355024
start iteration
done sorting
end inner loop
Iteration 4, inertia 144.11933105355024
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 155.32881279549147
start iteration
done sorting
end inner loop
Iteration 1, inertia 148.9184448303734
start iteration
done sorting
end inner loop
Iteration 2, inertia 147.9452905799804
start iteration
done sorting
end inner loop
Iteration 3, inertia 147.9452905799804
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner l

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0371s.) Setting batch_size=10.
[Parallel(n_jobs=20)]: Batch computation too fast (0.0371s.) Setting batch_size=106.
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 434 out of 1600 | elapsed:    7.6s remaining:   20.4s
[Parallel(n_jobs=20)]: Done 1600 out of 1600 | elapsed:   23.4s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0287s.) Setting batch_size=12.


******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17150798790412242
feature index: 2 | L2 norm: 0.17150798790412242
0.0
0.06847590909173441
feature index: 4 | L2 norm: 0.06847590909173441
0.030613638060943388
feature index: 5 | L2 norm: 0.030613638060943388
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.024950310843724528
feature index: 14 | L2 norm: 0.024950310843724528
0.0
0.04361108049900838
feature index: 16 | L2 norm: 0.04361108049900838
0.0064768708433381224
feature index: 17 | L2 norm: 0.0064768708433381224
0.0
0.030194036632431875
feature index: 19 | L2 norm: 0.030194036632431875
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0416374726689765
feature index: 29 | L2 norm: 0.0416374726689765
0.11070466146080334
feature index: 30 | L2 norm: 0.11070466146080334
0.0
0.02901884838855065
feature index: 32 | L2 norm: 0.02901884838855065
0.0
0.0705879418871024
feature index: 34 | L2 norm: 0.0705879418871024
0.0
0.0
0.0
0.0
0.0
0.1267212641293222
feature index: 40 |

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:    4.0s
[Parallel(n_jobs=20)]: Done 1024 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Done 1252 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done 1480 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 1732 tasks      | elapsed:    7.4s
[Parallel(n_jobs=20)]: Done 1984 tasks      | elapsed:    8.3s
[Parallel(n_jobs=20)]: Done 2260 tasks      | elapsed:    9.2s
[Parallel(n_jobs=20)]: Done 2536 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done 2836 tasks      | elapsed:   11.1s


******** SVM best params **********
{'C': 0.0001, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.9473684210526315

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.89      0.94         9
           1       0.91      1.00      0.95        10

   micro avg       0.95      0.95      0.95        19
   macro avg       0.95      0.94      0.95        19
weighted avg       0.95      0.95      0.95        19

Number of clusters: 12
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 144.88159395501862
start iteration
done sorting
end inner loop
Iteration 1, inertia 142.71622113000484
start iteration
done sorting
end inner loop
Iteration 2, inertia 140.50910674819534
start iteration
done sorting
end inner loop
Iteration 3, inertia 139.86595634371272
start iteration
done sorting
end inner loop
Iteration 4, inertia 139.86595634371272
center shift 0.000000e+00 within tolerance 

...

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0653s.) Setting batch_size=6.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done  70 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 148 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0304s.) Setting batch_size=3.
[Parallel(n_jobs=20)]: Done 238 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 322 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0021s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done 409 tasks      | elapsed:    9.7s
[Parallel(n_jobs=20)]: Done 456 tasks      | elapsed:   12.5s
[Parallel(n_jobs=20)]: Done 479 tasks      | elapsed:   13.6s
[Parallel(n_jobs=20)]: Done 498 tasks      | elapsed:   14.0s
[Parall

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.1668560653063677
feature index: 2 | L2 norm: 0.1668560653063677
0.0
0.05943356303183214
feature index: 4 | L2 norm: 0.05943356303183214
0.04716806369940713
feature index: 5 | L2 norm: 0.04716806369940713
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.014162087607959156
feature index: 14 | L2 norm: 0.014162087607959156
0.0
0.05372296251617076
feature index: 16 | L2 norm: 0.05372296251617076
0.0
0.0
0.021783107967593647
feature index: 19 | L2 norm: 0.021783107967593647
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.04192720828226071
feature index: 29 | L2 norm: 0.04192720828226071
0.10660933234868661
feature index: 30 | L2 norm: 0.10660933234868661
0.0
0.03493872211370993
feature index: 32 | L2 norm: 0.03493872211370993
0.0
0.07554513990354594
feature index: 34 | L2 norm: 0.07554513990354594
0.0
0.0
0.0
0.0
0.0
0.12697513192920096
feature index: 40 | L2 norm: 0.12697513192920096
0.0
0.0
0.0
0.0
0.0
0.3092786211321724

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    4.0s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.9s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.5s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    9.1s
[Parallel(n_jobs=20)]: Done 2620 tasks      | elapsed:   10.1s


******** SVM best params **********
{'C': 0.1, 'gamma': 1000, 'kernel': 'sigmoid'}

***** accuracy *****
0.8947368421052632

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.78      0.88         9
           1       0.83      1.00      0.91        10

   micro avg       0.89      0.89      0.89        19
   macro avg       0.92      0.89      0.89        19
weighted avg       0.91      0.89      0.89        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed:  9.3min finished


**Freesurfer dataset MCI vs AD**

In [17]:
traindf = pd.read_csv('C:\\Users\\Administrator\\no_header_shuffled_train_dataset_full_features_AD_vs_MCI.txt',sep="\t",header=None)
testdf = pd.read_csv('C:\\Users\\Administrator\\no_header_test_dataset_full_features_AD_vs_MCI.txt',sep="\t",header=None)

In [18]:
df_features = traindf.iloc[:,:91]

df_positive_features = traindf[traindf[91] == 1]
df_positive_features = df_positive_features.iloc[:,:91]

df_negative_features = traindf[traindf[91] == -1]
df_negative_features = df_negative_features.iloc[:,:91]

In [0]:
kmeans_clusters = [2,3,4,5,6]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 2
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 342.6921092176821
start iteration
done sorting
end inner loop
Iteration 1, inertia 310.0301727944895
start iteration
done sorting
end inner loop
Iteration 2, inertia 305.59367466897845
start iteration
done sorting
end inner loop
Iteration 3, inertia 304.36033488848057
start iteration
done sorting
end inner loop
Iteration 4, inertia 303.5804434532913
start iteration
done sorting
end inner loop
Iteration 5, inertia 303.5804434532913
center shift 0.000000e+00 within tolerance 6.513796e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 304.66989026915553
start iteration
done sorting
end inner loop
Iteration 1, inertia 303.9822345211105
start iteration
done sorting
end inner loop
Iteration 2, inertia 303.5804434532913
start iteration
done sorting
end inner loop
Iteration 3, inertia 303.5804434532913
center shift 0.000000e+00 within toleranc

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    9.6s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   10.0s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   10.3s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   10.7s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   11.1s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   11.5s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   11.9s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   12.1s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   12.5s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   13.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   13.9s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   14.6s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.036553857907644816
feature index: 0 | L2 norm: 0.036553857907644816
0.0
0.1512926675586825
feature index: 2 | L2 norm: 0.1512926675586825
0.0
0.0
0.0
0.03129625986470538
feature index: 6 | L2 norm: 0.03129625986470538
0.0
0.0
0.0
0.0
0.07482157833975796
feature index: 11 | L2 norm: 0.07482157833975796
0.0
0.0
0.0
0.0
0.09730473260887665
feature index: 16 | L2 norm: 0.09730473260887665
0.00534188330023555
feature index: 17 | L2 norm: 0.00534188330023555
0.0
0.0
0.0
0.0
0.045481456235264714
feature index: 22 | L2 norm: 0.045481456235264714
0.0
0.028498083135641727
feature index: 24 | L2 norm: 0.028498083135641727
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.023495203428434964
feature index: 32 | L2 norm: 0.023495203428434964
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03159995049860799
feature index: 41 | L2 norm: 0.03159995049860799
0.0
0.0
0.0
0.0
0.0
0.0
0.04546858820126026
feature index: 48 | L2 norm: 0.04546858820126026
0.

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    5.2s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.9s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.7s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.3s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    8.1s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    9.0s
[Parallel(n_jobs=20)]: Done 2620 tasks      | elapsed:   10.0s


******** SVM best params **********
{'C': 0.1, 'gamma': 100, 'kernel': 'sigmoid'}

***** accuracy *****
0.5789473684210527

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.56      0.56      0.56         9
           1       0.60      0.60      0.60        10

   micro avg       0.58      0.58      0.58        19
   macro avg       0.58      0.58      0.58        19
weighted avg       0.58      0.58      0.58        19

Number of clusters: 3
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 300.7069791909862
start iteration
done sorting
end inner loop
Iteration 1, inertia 291.7933937137642
start iteration
done sorting
end inner loop
Iteration 2, inertia 282.8459881018341
start iteration
done sorting
end inner loop
Iteration 3, inertia 280.01153490131685
start iteration
done sorting
end inner loop
Iteration 4, inertia 277.73862135438935
start iteration
done sorting
end inner loop
Itera

...

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0680s.) Setting batch_size=4.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  60 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 112 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 172 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 232 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 300 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 368 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Batch computation too slow (2.1800s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done 444 tasks      | elapsed:    9.1s
[Parallel(n_jobs=20)]: Done 516 tasks      | elapsed:    9.8s
[Parallel(n_jobs=20)]: Done 568 tasks      | elapsed:   10.4s
[Parallel(n_jobs=20)]: Batch

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.04041471892127525
feature index: 0 | L2 norm: 0.04041471892127525
0.0
0.1266559684157653
feature index: 2 | L2 norm: 0.1266559684157653
0.0
0.0
0.0
0.029325209791759718
feature index: 6 | L2 norm: 0.029325209791759718
0.0
0.0
0.0
0.0
0.07178325312544405
feature index: 11 | L2 norm: 0.07178325312544405
0.0
0.0
0.0
0.0
0.06834725308270131
feature index: 16 | L2 norm: 0.06834725308270131
0.0007820669378302271
feature index: 17 | L2 norm: 0.0007820669378302271
0.0
0.0
0.0
0.0
0.04429422504732902
feature index: 22 | L2 norm: 0.04429422504732902
0.0
0.0528780229197446
feature index: 24 | L2 norm: 0.0528780229197446
0.0
0.0
0.0
0.013894246079618778
feature index: 28 | L2 norm: 0.013894246079618778
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.043631717864622936
feature index: 41 | L2 norm: 0.043631717864622936
0.0
0.0
0.0
0.001131137178193304
feature index: 45 | L2 norm: 0.001131137178193304
0.0
0.0
0.0


[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.6s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.3s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    8.0s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    9.0s
[Parallel(n_jobs=20)]: Done 2620 tasks      | elapsed:    9.8s


******** SVM best params **********
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

***** accuracy *****
0.6842105263157895

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.64      0.78      0.70         9
           1       0.75      0.60      0.67        10

   micro avg       0.68      0.68      0.68        19
   macro avg       0.69      0.69      0.68        19
weighted avg       0.70      0.68      0.68        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed: 15.0min finished


In [0]:
kmeans_clusters = [7,8,9,10,11]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 7
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 178.84900477489168
start iteration
done sorting
end inner loop
Iteration 1, inertia 168.46286231051076
start iteration
done sorting
end inner loop
Iteration 2, inertia 167.15630465229998
start iteration
done sorting
end inner loop
Iteration 3, inertia 166.81911924408877
start iteration
done sorting
end inner loop
Iteration 4, inertia 166.55690302889127
start iteration
done sorting
end inner loop
Iteration 5, inertia 166.55690302889127
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 180.10209191435135
start iteration
done sorting
end inner loop
Iteration 1, inertia 171.22836743341264
start iteration
done sorting
end inner loop
Iteration 2, inertia 171.22836743341264
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    8.1s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    9.6s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   10.0s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   10.5s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   11.1s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   11.5s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   12.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   12.9s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   13.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   13.7s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   14.7s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17257115467840645
feature index: 2 | L2 norm: 0.17257115467840645
0.0
0.057319877593947466
feature index: 4 | L2 norm: 0.057319877593947466
0.042720998250052294
feature index: 5 | L2 norm: 0.042720998250052294
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.004774391161581537
feature index: 14 | L2 norm: 0.004774391161581537
0.0
0.0501139760756843
feature index: 16 | L2 norm: 0.0501139760756843
0.0
0.0
0.02015342608181814
feature index: 19 | L2 norm: 0.02015342608181814
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.04091336933441739
feature index: 29 | L2 norm: 0.04091336933441739
0.11218312742280925
feature index: 30 | L2 norm: 0.11218312742280925
0.0
0.022111794509153248
feature index: 32 | L2 norm: 0.022111794509153248
0.0
0.07311590813918949
feature index: 34 | L2 norm: 0.07311590813918949
0.0
0.0
0.0
0.0
0.0
0.12882169349965855
feature index: 40 | L2 norm: 0.12882169349965855
0.0
0.0
0.0
0.0
0.0
0.313377058082

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.8s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.8s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.5s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    8.4s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    9.2s
[

******** SVM best params **********
{'C': 0.1, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.8947368421052632

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.78      0.88         9
           1       0.83      1.00      0.91        10

   micro avg       0.89      0.89      0.89        19
   macro avg       0.92      0.89      0.89        19
weighted avg       0.91      0.89      0.89        19

Number of clusters: 8
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 168.35038562119092
start iteration
done sorting
end inner loop
Iteration 1, inertia 162.43135780476172
start iteration
done sorting
end inner loop
Iteration 2, inertia 161.8934694376739
start iteration
done sorting
end inner loop
Iteration 3, inertia 161.43534368174164
start iteration
done sorting
end inner loop
Iteration 4, inertia 161.43534368174164
center shift 0.000000e+00 within tolerance 4.810

...

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done  80 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 184 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0365s.) Setting batch_size=4.
[Parallel(n_jobs=20)]: Done 304 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 412 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 480 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Batch computation too slow (2.5289s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done 572 tasks      | elapsed:   11.7s
[Parallel(n_jobs=20)]: Batch computation too slow (4.2575s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done 668 tasks      | elapsed:   13.3s
[Parallel(n_jobs=20)]: Done 707 tasks      | elapsed:   13.8s
[Parallel(n_jobs=20)]: Done 734 tasks      | elapsed:   14.0s
[Parall

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17150798790412242
feature index: 2 | L2 norm: 0.17150798790412242
0.0
0.06847590909173441
feature index: 4 | L2 norm: 0.06847590909173441
0.030613638060943388
feature index: 5 | L2 norm: 0.030613638060943388
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.024950310843724528
feature index: 14 | L2 norm: 0.024950310843724528
0.0
0.04361108049900838
feature index: 16 | L2 norm: 0.04361108049900838
0.0064768708433381224
feature index: 17 | L2 norm: 0.0064768708433381224
0.0
0.030194036632431875
feature index: 19 | L2 norm: 0.030194036632431875
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0416374726689765
feature index: 29 | L2 norm: 0.0416374726689765
0.11070466146080334
feature index: 30 | L2 norm: 0.11070466146080334
0.0
0.02901884838855065
feature index: 32 | L2 norm: 0.02901884838855065
0.0
0.0705879418871024
feature index: 34 | L2 norm: 0.0705879418871024
0.0
0.0
0.0
0.0
0.0
0.1267212641293222
feature index: 40 |

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0371s.) Setting batch_size=10.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.2s
[Parallel(n_jobs=20)]: Done 1890 tasks      

******** SVM best params **********
{'C': 0.0001, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.9473684210526315

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.89      0.94         9
           1       0.91      1.00      0.95        10

   micro avg       0.95      0.95      0.95        19
   macro avg       0.95      0.94      0.95        19
weighted avg       0.95      0.95      0.95        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed:  8.9min finished


In [0]:
kmeans_clusters = [12,13,14,15,16]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


Number of clusters: 12
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 144.88159395501862
start iteration
done sorting
end inner loop
Iteration 1, inertia 142.71622113000484
start iteration
done sorting
end inner loop
Iteration 2, inertia 140.50910674819534
start iteration
done sorting
end inner loop
Iteration 3, inertia 139.86595634371272
start iteration
done sorting
end inner loop
Iteration 4, inertia 139.86595634371272
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 152.0200905518627
start iteration
done sorting
end inner loop
Iteration 1, inertia 143.83261599490731
start iteration
done sorting
end inner loop
Iteration 2, inertia 143.83261599490731
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 149.68140187932556
start iteration
done sorting
end inner

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.1321s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  50 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done  76 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 106 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 136 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 170 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 204 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 242 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 322 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 364 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 410 tasks      | elap

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17150798790412242
feature index: 2 | L2 norm: 0.17150798790412242
0.0
0.06847590909173441
feature index: 4 | L2 norm: 0.06847590909173441
0.030613638060943388
feature index: 5 | L2 norm: 0.030613638060943388
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.024950310843724528
feature index: 14 | L2 norm: 0.024950310843724528
0.0
0.04361108049900838
feature index: 16 | L2 norm: 0.04361108049900838
0.0064768708433381224
feature index: 17 | L2 norm: 0.0064768708433381224
0.0
0.030194036632431875
feature index: 19 | L2 norm: 0.030194036632431875
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0416374726689765
feature index: 29 | L2 norm: 0.0416374726689765
0.11070466146080334
feature index: 30 | L2 norm: 0.11070466146080334
0.0
0.02901884838855065
feature index: 32 | L2 norm: 0.02901884838855065
0.0
0.0705879418871024
feature index: 34 | L2 norm: 0.0705879418871024
0.0
0.0
0.0
0.0
0.0
0.1267212641293222
feature index: 40 |

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 110 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 292 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 502 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0097s.) Setting batch_size=7.
[Parallel(n_jobs=20)]: Done 712 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 950 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 1132 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 1265 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done 1398 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done 1545 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done 1692 tasks      | elapsed:    6.0s
[Parallel(n_jobs=20)]: Done 1853 tasks      | elapsed:    6.6s
[Parallel(n_jobs=20)]: Done 2014 tasks      | elapsed:    7.2s
[Parallel(n_jobs=20)]: Done 2189 tasks    

...

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0884s.) Setting batch_size=4.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  60 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 112 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 172 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Done 232 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 300 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 368 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Batch computation too slow (2.1208s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done 444 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done 512 tasks      | elapsed:   10.8s
[Parallel(n_jobs=20)]: Done 564 tasks      | elapsed:   11.2s
[Parallel(n_jobs=20)]: Batch

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.17312475795160653
feature index: 2 | L2 norm: 0.17312475795160653
0.0
0.0533981578726999
feature index: 4 | L2 norm: 0.0533981578726999
0.043789951387613324
feature index: 5 | L2 norm: 0.043789951387613324
0.0
0.0
0.004318508179696125
feature index: 8 | L2 norm: 0.004318508179696125
0.0
0.0
0.0
0.0
0.0
0.028517777057763247
feature index: 14 | L2 norm: 0.028517777057763247
0.0
0.033841290960184525
feature index: 16 | L2 norm: 0.033841290960184525
0.008366742290015949
feature index: 17 | L2 norm: 0.008366742290015949
0.0
0.01603471882474623
feature index: 19 | L2 norm: 0.01603471882474623
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03641803277325595
feature index: 29 | L2 norm: 0.03641803277325595
0.106264011618771
feature index: 30 | L2 norm: 0.106264011618771
0.0
0.03623459192970188
feature index: 32 | L2 norm: 0.03623459192970188
0.0
0.07539313536844039
feature index: 34 | L2 norm: 0.07539313536844

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done  90 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 370 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 520 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 690 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 860 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 1050 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done 1240 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:    5.6s
[Parallel(n_jobs=20)]: Done 1660 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 1890 tasks      | elapsed:    7.0s
[Parallel(n_jobs=20)]: Done 2120 tasks      | elapsed:    7.7s
[Parallel(n_jobs=20)]: Done 2370 tasks      | elapsed:    8.5s
[Parallel(n_jobs=20)]: Done 2620 tasks      | elapsed:    9.4s


******** SVM best params **********
{'C': 0.1, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.7894736842105263

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.78      0.78      0.78         9
           1       0.80      0.80      0.80        10

   micro avg       0.79      0.79      0.79        19
   macro avg       0.79      0.79      0.79        19
weighted avg       0.79      0.79      0.79        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed: 11.0min finished


In [0]:
kmeans_clusters = [17,18,19,20]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 17
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 125.1212755367462
start iteration
done sorting
end inner loop
Iteration 1, inertia 121.8488505055357
start iteration
done sorting
end inner loop
Iteration 2, inertia 121.8488505055357
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 122.80597216752575
start iteration
done sorting
end inner loop
Iteration 1, inertia 119.25647630332816
start iteration
done sorting
end inner loop
Iteration 2, inertia 118.67197625239362
start iteration
done sorting
end inner loop
Iteration 3, inertia 118.67197625239362
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 126.05519627248496
start iteration
done sorting
end inner loop
Iteration 1, inertia 124.12096890314803
start iteration
done sorting
end inner l

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0548s.) Setting batch_size=6.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done  70 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 148 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Batch computation too slow (2.0386s.) Setting batch_size=3.
[Parallel(n_jobs=20)]: Done 238 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 328 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Batch computation too slow (4.8622s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done 415 tasks      | elapsed:    8.5s
[Parallel(n_jobs=20)]: Done 466 tasks      | elapsed:   10.3s
[Parallel(n_jobs=20)]: Done 491 tasks      | elapsed: 

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.16934786484788242
feature index: 2 | L2 norm: 0.16934786484788242
0.0
0.05328286948329925
feature index: 4 | L2 norm: 0.05328286948329925
0.05615503778370375
feature index: 5 | L2 norm: 0.05615503778370375
0.0
0.0
0.004448513814192797
feature index: 8 | L2 norm: 0.004448513814192797
0.0
0.0
0.0
0.0
0.0
0.02203338253367689
feature index: 14 | L2 norm: 0.02203338253367689
0.0
0.05759829544815424
feature index: 16 | L2 norm: 0.05759829544815424
0.0
0.0
0.029736691106070388
feature index: 19 | L2 norm: 0.029736691106070388
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.045582280764323434
feature index: 29 | L2 norm: 0.045582280764323434
0.10250595439281428
feature index: 30 | L2 norm: 0.10250595439281428
0.0
0.03164528554380511
feature index: 32 | L2 norm: 0.03164528554380511
0.0
0.07936278356418405
feature index: 34 | L2 norm: 0.07936278356418405
0.0
0.0
0.0
0.0
0.0
0.12520612839708894
feature index: 40 |

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 100 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 256 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 616 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 820 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 1024 tasks      | elapsed:    4.3s
[Parallel(n_jobs=20)]: Done 1252 tasks      | elapsed:    5.2s
[Parallel(n_jobs=20)]: Done 1480 tasks      | elapsed:    5.8s
[Parallel(n_jobs=20)]: Done 1732 tasks      | elapsed:    6.8s
[Parallel(n_jobs=20)]: Done 1984 tasks      | elapsed:    7.6s
[Parallel(n_jobs=20)]: Done 2260 tasks      | elapsed:    8.6s
[Parallel(n_jobs=20)]: Done 2536 tasks      | elapsed:    9.5s
[Parallel(n_jobs=20)]: Done 2836 tasks      | elapsed:   10.5s
[Parallel(n_jobs=20)]: Done 3136 tasks      | elapsed:   11.4s

******** SVM best params **********
{'C': 0.0001, 'gamma': 10, 'kernel': 'sigmoid'}

***** accuracy *****
0.9473684210526315

***** Classification report ********
              precision    recall  f1-score   support

          -1       1.00      0.89      0.94         9
           1       0.91      1.00      0.95        10

   micro avg       0.95      0.95      0.95        19
   macro avg       0.95      0.94      0.95        19
weighted avg       0.95      0.95      0.95        19

Number of clusters: 18
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 121.08017198605654
start iteration
done sorting
end inner loop
Iteration 1, inertia 117.80774695484607
start iteration
done sorting
end inner loop
Iteration 2, inertia 117.80774695484607
center shift 0.000000e+00 within tolerance 4.810248e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 124.08455733718917
start iteration
done sorting
end inner loop
Ite

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0633s.) Setting batch_size=6.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done  70 tasks      | elapsed:    1.7s
[Parallel(n_jobs=20)]: Done 148 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Batch computation too slow (2.1011s.) Setting batch_size=3.
[Parallel(n_jobs=20)]: Done 238 tasks      | elapsed:    4.0s
[Parallel(n_jobs=20)]: Done 328 tasks      | elapsed:    4.7s
[Parallel(n_jobs=20)]: Batch computation too slow (5.2149s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done 415 tasks      | elapsed:    8.9s
[Parallel(n_jobs=20)]: Done 466 tasks      | elapsed:   10.9s
[Parallel(n_jobs=20)]: Done 491 tasks      | elapsed:   11.3s
[Parallel(n_jobs=20)]: Done 512 tasks      | elapsed:   11.7s
[Parall

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.0
0.0
0.1754874003247337
feature index: 2 | L2 norm: 0.1754874003247337
0.0
0.06205085571069682
feature index: 4 | L2 norm: 0.06205085571069682
0.05875905831123417
feature index: 5 | L2 norm: 0.05875905831123417
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.010750731032553397
feature index: 14 | L2 norm: 0.010750731032553397
0.0
0.04826986916854876
feature index: 16 | L2 norm: 0.04826986916854876
0.0
0.0
0.008397901835129562
feature index: 19 | L2 norm: 0.008397901835129562
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.039212269412451914
feature index: 29 | L2 norm: 0.039212269412451914
0.11875367017049682
feature index: 30 | L2 norm: 0.11875367017049682
0.0
0.022256996156683567
feature index: 32 | L2 norm: 0.022256996156683567
0.0
0.08056893700587786
feature index: 34 | L2 norm: 0.08056893700587786
0.0
0.0
0.0
0.0
0.0
0.13539284430725468
feature index: 40 | L2 norm: 0.13539284430725468
0.0
0.0
0.0
0.0
0.0
0.302807240591

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 110 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 292 tasks      | elapsed:    2.4s
[Parallel(n_jobs=20)]: Done 502 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done 712 tasks      | elapsed:    3.8s
[Parallel(n_jobs=20)]: Done 950 tasks      | elapsed:    4.6s
[Parallel(n_jobs=20)]: Done 1188 tasks      | elapsed:    5.3s
[Parallel(n_jobs=20)]: Done 1454 tasks      | elapsed:    6.1s
[Parallel(n_jobs=20)]: Done 1720 tasks      | elapsed:    6.9s
[Parallel(n_jobs=20)]: Done 2014 tasks      | elapsed:    7.8s
[Parallel(n_jobs=20)]: Done 2308 tasks      | elapsed:    8.7s
[Parallel(n_jobs=20)]: Done 2630 tasks      | elapsed:    9.7s
[Parallel(n_jobs=20)]: Done 2952 tasks      | elapsed:   10.7s
[Parallel(n_jobs=20)]: Done 3302 tasks      | elapsed:   11.7s


In [19]:
kmeans_clusters = [18,19,20]
for n_cluster in kmeans_clusters:
  print("Number of clusters: "+str(n_cluster))
  kmeans = fit_kmeans(df_features,n_cluster)
  positive_features_subclass = kmeans.predict(df_positive_features)
  negative_features_subclass = kmeans.predict(df_negative_features)
  zeros = prepare_multitask_matrix(n_cluster,positive_features_subclass,negative_features_subclass)
  multi_task_Y = prepare_multitask_output(traindf[91],zeros,n_cluster)
  mtl_model = fit_multi_task_lasso(df_features.values,multi_task_Y,do_cv=True)
  best_params = get_lasso_best_params(mtl_model,'mtl_lasso')
  
  print("******* Lasso best params *********")
  print(best_params)
  print()

  scaler = StandardScaler()
  normalized_train_data = scaler.fit_transform(df_features)
  mtl_coefficients = fit_multi_task_lasso(normalized_train_data,multi_task_Y,do_cv=False,best_params = best_params)
  
  reduced_features_list = get_reduced_feature_list(mtl_coefficients)
  
  print("****** reduced feature list ********")
  print(reduced_features_list)
  print()
  
  reduced_features_train = traindf[reduced_features_list]
  reduced_features_test = testdf[reduced_features_list]
  
  svm_model_cv = fit_svm(reduced_features_train,traindf[[91]],cv=True)
  
  svm_best_params = get_svm_best_params(svm_model_cv,'classifier')
  
  print("******** SVM best params **********")
  print(svm_best_params)
  print()
  
  svm_model = fit_svm(reduced_features_train,traindf[[91]],cv=False,best_params=svm_best_params)
  svm_predict = predict_svm(svm_model,reduced_features_test)
  
  accuracy = accuracy_score(testdf[[91]].values,svm_predict)
  report = classification_report(testdf[[91]].values,svm_predict)
  
  print("***** accuracy *****")
  print(accuracy)
  
  print()
  
  print("***** Classification report ********")
  print(report)
  
  

Number of clusters: 18
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 141.52536898007065
start iteration
done sorting
end inner loop
Iteration 1, inertia 140.93993278347563
start iteration
done sorting
end inner loop
Iteration 2, inertia 140.93993278347563
center shift 0.000000e+00 within tolerance 6.513796e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 146.57359941400904
start iteration
done sorting
end inner loop
Iteration 1, inertia 140.24304661453363
start iteration
done sorting
end inner loop
Iteration 2, inertia 138.2245605961764
start iteration
done sorting
end inner loop
Iteration 3, inertia 137.44211432520143
start iteration
done sorting
end inner loop
Iteration 4, inertia 137.44211432520143
center shift 0.000000e+00 within tolerance 6.513796e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 146.78488910244565
start iteration
done sorting
end inner

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Batch computation too fast (0.1977s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0931s.) Setting batch_size=8.
[Parallel(n_jobs=20)]: Done 112 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 145 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 229 tasks      | elapsed:    3.8s
[Parallel(n_jobs=20)]: Done 374 tasks      | elapsed:    4.4s
[Parallel(n_j

******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.04520090897803828
feature index: 0 | L2 norm: 0.04520090897803828
0.0
0.15036604324458328
feature index: 2 | L2 norm: 0.15036604324458328
0.0
0.0
0.0
0.046573216815302715
feature index: 6 | L2 norm: 0.046573216815302715
0.0
0.0
0.0
0.0
0.047773143441544444
feature index: 11 | L2 norm: 0.047773143441544444
0.0
0.0
0.0
0.0
0.05875393398690265
feature index: 16 | L2 norm: 0.05875393398690265
0.0
0.0
0.0
0.0
0.0
0.05504791060719341
feature index: 22 | L2 norm: 0.05504791060719341
0.0028043986929863866
feature index: 23 | L2 norm: 0.0028043986929863866
0.04939524041422458
feature index: 24 | L2 norm: 0.04939524041422458
0.0
0.0
0.0
0.012295234563885987
feature index: 28 | L2 norm: 0.012295234563885987
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.006196545260225913
feature index: 41 | L2 norm: 0.006196545260225913
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.11342866309721181
feature index: 52 | L2 norm: 

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 472 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 832 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 1192 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 1600 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 2008 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done 2464 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 2920 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 3424 tasks      | elapsed:    3.2s
[Parallel(n_jobs=20)]: Done 3928 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 4480 tasks      | elapsed:    4.0s
[Parallel(n_jobs=20)]: Done 5032 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 5632 tasks      | elapsed:    4.8

******** SVM best params **********
{'C': 0.001, 'gamma': 1000, 'kernel': 'sigmoid'}

***** accuracy *****
0.42105263157894735

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.33      0.22      0.27         9
           1       0.46      0.60      0.52        10

   micro avg       0.42      0.42      0.42        19
   macro avg       0.40      0.41      0.39        19
weighted avg       0.40      0.42      0.40        19

Number of clusters: 19
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 135.83588205110175
start iteration
done sorting
end inner loop
Iteration 1, inertia 135.83588205110175
center shift 0.000000e+00 within tolerance 6.513796e-06
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 144.03396888881338
start iteration
done sorting
end inner loop
Iteration 1, inertia 140.21586368935982
start iteration
done sorting
end inner loop
I

...

[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 472 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Batch computation too slow (2.2352s.) Setting batch_size=12.
[Parallel(n_jobs=20)]: Done 832 tasks      | elapsed:    7.8s
[Parallel(n_jobs=20)]: Done 1600 out of 1600 | elapsed:    9.6s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0156s.) Setting batch_size=24.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s


******* Lasso best params *********
{'alpha': 0.1, 'max_iter': 15000, 'tol': 0.01}

0.03394473392189711
feature index: 0 | L2 norm: 0.03394473392189711
0.0
0.13451559045909875
feature index: 2 | L2 norm: 0.13451559045909875
0.0
0.0
0.0
0.049510368486544304
feature index: 6 | L2 norm: 0.049510368486544304
0.0
0.0
0.0
0.0
0.060292723622363056
feature index: 11 | L2 norm: 0.060292723622363056
0.0
0.0
0.0
0.0
0.05596338435978845
feature index: 16 | L2 norm: 0.05596338435978845
0.03400916712560465
feature index: 17 | L2 norm: 0.03400916712560465
0.0
0.0
0.0
0.0
0.052773146797680276
feature index: 22 | L2 norm: 0.052773146797680276
0.0
0.046527267330182275
feature index: 24 | L2 norm: 0.046527267330182275
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.008828577578976798
feature index: 41 | L2 norm: 0.008828577578976798
0.0057081178067385895
feature index: 42 | L2 norm: 0.0057081178067385895
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.12093043576328091
feature index: 52 | L2 norm

[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 472 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 832 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 1192 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 1600 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 2008 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 2464 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 2920 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 3424 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 3928 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done 4480 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 5032 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done 5632 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 6232 tasks      | elapsed:    5.

******** SVM best params **********
{'C': 0.0001, 'gamma': 1000, 'kernel': 'sigmoid'}

***** accuracy *****
0.42105263157894735

***** Classification report ********
              precision    recall  f1-score   support

          -1       0.33      0.22      0.27         9
           1       0.46      0.60      0.52        10

   micro avg       0.42      0.42      0.42        19
   macro avg       0.40      0.41      0.39        19
weighted avg       0.40      0.42      0.40        19



[Parallel(n_jobs=20)]: Done 19200 out of 19200 | elapsed:  3.8min finished
